In [ ]:
Das notebook prüft den compoud wert 

In [11]:
import re
import pandas as pd
import gensim
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji
import demoji
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
# read csv 
df = pd.read_csv('Data/aftercorona.csv')
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site
0,2.730776e+07,50931,AppelquistP,Damn I thought I was finally in the #Matrix or...,0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode from #blackmirror gets ...,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
2,2.370116e+09,115767,Niggvtivity,I've just watched episode S03 | E05 of Black M...,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
3,4.817271e+07,4971,DystopicRedhead,The perfect description of the #Tories and the...,4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
4,1.402080e+18,13,smash_media_us,ã€ #ONEUS Ã— #smash. ã€‘\r\n\r\nONEUS is on n...,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json


In [13]:
df.columns

Index(['user_id', 'user_statuses', 'name', 'tweet', 'retweets', 'location',
       'created', 'followers', 'is_user_verified', 'favorite_count',
       'in_reply_to_status_id', 'keyword', 'language', 'site'],
      dtype='object')

In [14]:
df.shape

(1735, 14)

In [15]:
def clean_complete_for_vader(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}
    tweet = tweet.str.replace(r':-\)', ' smile')
    tweet = tweet.str.replace(r':-\(', ' sad')
    tweet = tweet.str.replace(r':-\/', ' confused')
    tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')
    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    tweet = tweet.str.replace('ℐℓ٥ﻻﻉ√٥υ', '')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [16]:
# apply cleaning for vader
df['tweet_vader'] = clean_complete_for_vader(df.tweet)

<ipython-input-15-a9104f2eb04e>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
<ipython-input-15-a9104f2eb04e>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\)', ' smile')
<ipython-input-15-a9104f2eb04e>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\(', ' sad')
<ipython-input-15-a9104f2eb04e>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\/', ' confused')


In [17]:
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site,tweet_vader
0,2.730776e+07,50931,AppelquistP,Damn I thought I was finally in the #Matrix or...,0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,Damn thought finally #Matrix #blackmirror #Ali...
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode from #blackmirror gets ...,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,â€œNosediveâ€ episode #blackmirror gets close...
2,2.370116e+09,115767,Niggvtivity,I've just watched episode S03 | E05 of Black M...,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,I've watched episode Black Mirror! #BlackMirro...
3,4.817271e+07,4971,DystopicRedhead,The perfect description of the #Tories and the...,4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,The perfect description #Tories voters favouri...
4,1.402080e+18,13,smash_media_us,ã€ #ONEUS Ã— #smash. ã€‘\r\n\r\nONEUS is on n...,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,ã€ #ONEUS #smash. ã€‘ ONEUS smash.â£ï¸ New ...


In [18]:
df.shape

(1735, 15)

In [20]:
#Vader sentiment analyzer
#The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).
sid_obj = SentimentIntensityAnalyzer()
df['compound'] = [sid_obj.polarity_scores(c)['compound'] for c in df['tweet_vader']]

In [21]:
print(df[['tweet_vader', 'compound']].describe())

          compound
count  1735.000000
mean      0.088540
std       0.438813
min      -0.963300
25%       0.000000
50%       0.000000
75%       0.440400
max       0.970500


In [22]:
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site,tweet_vader,compound
0,2.730776e+07,50931,AppelquistP,Damn I thought I was finally in the #Matrix or...,0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,Damn thought finally #Matrix #blackmirror #Ali...,0.0000
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode from #blackmirror gets ...,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,â€œNosediveâ€ episode #blackmirror gets close...,0.0000
2,2.370116e+09,115767,Niggvtivity,I've just watched episode S03 | E05 of Black M...,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,I've watched episode Black Mirror! #BlackMirro...,0.0000
3,4.817271e+07,4971,DystopicRedhead,The perfect description of the #Tories and the...,4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,The perfect description #Tories voters favouri...,0.5719
4,1.402080e+18,13,smash_media_us,ã€ #ONEUS Ã— #smash. ã€‘\r\n\r\nONEUS is on n...,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json,ã€ #ONEUS #smash. ã€‘ ONEUS smash.â£ï¸ New ...,0.2003


In [23]:
# save result
df.to_csv('Data/Vader_Eval_Original.csv', sep=',', header=True)